# Project | NCAA Website Scraping
### Emily Nell, Chaitanya Patel, Emily Sanders

---

In [58]:
import pandas as pd
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By # used to import different ways to access data in the XML or HTML file
from selenium.webdriver.chrome.service import Service # no longer need to download a driver file, use service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from urllib.parse import urljoin
from webdriver_manager.chrome import ChromeDriverManager # used to manage the Chrome driver to emulate a Chrome web browser

import time
import random

In [59]:
# URL of the College Football Win-Loss Records & Trends Website
url = 'https://betiq.teamrankings.com/college-football/betting-trends/win-loss-records/'

# Extract tables from the webpage
tables = pd.read_html(url)

# First table is one with data needed for project
df_win_loss_trends_24 = tables[0]

# Display the DataFrame
df_win_loss_trends_24

,Team,Win-Loss Record,Win %,MOV,ATS +/-
0,Oregon,13-1-0,92.9%,15.4,-2.3
1,Ohio St,14-2-0,87.5%,22.8,3.0
2,Notre Dame,14-2-0,87.5%,20.6,9.3
3,Boise St,12-2-0,85.7%,14.7,0.9
4,Army,12-2-0,85.7%,15.6,5.5
...,...,...,...,...,...
129,U Mass,2-10-0,16.7%,-13.0,-3.1
130,Florida St,2-10-0,16.7%,-12.6,-10.8
131,S Mississippi,1-11-0,8.3%,-22.5,-9.0
132,Purdue,1-11-0,8.3%,-24.2,-11.2


In [60]:
# MERGE 2024 NCAA STATS DATA
# ncaa_stats_2024 = df_Total_Offense.merge(df2, on=['Team',"G", "W-L", "Season"]).merge(df3, on=['Team',"G", "W-L", "Season"])

In [61]:
def scrape_metric_links_by_year(base_url, year, titles,
                                headless: bool = True,
                                timeout: int = 10) -> pd.DataFrame:
    """
    Scrape metric links for a specific year.
    """
    # --- setup driver ---
    opts = Options()
    if headless:
        opts.add_argument("--headless")
    driver = webdriver.Chrome(options=opts)

    # Construct the URL with the year parameter
    year_url = f"{base_url}&academic_year={year}"
    driver.get(year_url)

    wait = WebDriverWait(driver, timeout)
    # wait until the main table (or nav) is present
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "table")))

    records = []
    for title in titles:
        try:
            link = wait.until(
                EC.presence_of_element_located((By.LINK_TEXT, title))
            )
            raw_href = link.get_attribute("href")
            full_href = urljoin(year_url, raw_href)
        except TimeoutException:
            full_href = None
        records.append({"Title": title, "URL": full_href})

    driver.quit()
    return pd.DataFrame(records)


if __name__ == "__main__":
    BASE_URL = "https://stats.ncaa.org/rankings?sport_code=MFB&division=11"
    TITLES = [
        "Total Offense",
        "Rushing Offense",
        "Passing Offense",
        "Total Defense",
        "Rushing Defense",
        "Passing Yards Allowed",
        "Turnover Margin",
        "Fewest Penalties Per Game",
        "Fewest Penalty Yards Per Game",
        "Time of Possession",
    ]
    YEARS = [2025,2024, 2023, 2022]  # Add the years you want to scrape

    all_data = []
    for year in YEARS:
        df = scrape_metric_links_by_year(BASE_URL, year, TITLES, headless=False)
        df["Year"] = year
        all_data.append(df)

    final_df = pd.concat(all_data, ignore_index=True)
    print(final_df)

                            Title  \
0                   Total Offense   
1                 Rushing Offense   
2                 Passing Offense   
3                   Total Defense   
4                 Rushing Defense   
5           Passing Yards Allowed   
6                 Turnover Margin   
7       Fewest Penalties Per Game   
8   Fewest Penalty Yards Per Game   
9              Time of Possession   
10                  Total Offense   
11                Rushing Offense   
12                Passing Offense   
13                  Total Defense   
14                Rushing Defense   
15          Passing Yards Allowed   
16                Turnover Margin   
17      Fewest Penalties Per Game   
18  Fewest Penalty Yards Per Game   
19             Time of Possession   
20                  Total Offense   
21                Rushing Offense   
22                Passing Offense   
23                  Total Defense   
24                Rushing Defense   
25          Passing Yards Allowed   
2

In [62]:
final_df

,Title,URL,Year
0,Total Offense,https://stats.ncaa.org/rankings/national_ranki...,2025
1,Rushing Offense,https://stats.ncaa.org/rankings/national_ranki...,2025
2,Passing Offense,https://stats.ncaa.org/rankings/national_ranki...,2025
3,Total Defense,https://stats.ncaa.org/rankings/national_ranki...,2025
4,Rushing Defense,https://stats.ncaa.org/rankings/national_ranki...,2025
5,Passing Yards Allowed,https://stats.ncaa.org/rankings/national_ranki...,2025
6,Turnover Margin,https://stats.ncaa.org/rankings/national_ranki...,2025
7,Fewest Penalties Per Game,https://stats.ncaa.org/rankings/national_ranki...,2025
8,Fewest Penalty Yards Per Game,https://stats.ncaa.org/rankings/national_ranki...,2025
9,Time of Possession,https://stats.ncaa.org/rankings/national_ranki...,2025


In [63]:
final_df.to_csv("ncaa_metric_links.csv", index=False)

In [64]:
import time
import random
import requests
import pandas as pd

INPUT_CSV  = "ncaa_metric_links.csv"
OUTPUT_CSV = "ncaa_all_metrics_raw_data.csv"

# Load your list of metric titles and URLs
links_df = pd.read_csv(INPUT_CSV)

session = requests.Session()
session.headers.update({"User-Agent": "Mozilla/5.0"})

df_stats = []

for _, row in links_df.iterrows():
    title = row["Title"]
    url   = row["URL"]

    if pd.isna(url):
        print(f"Skipping {title!r}: no URL")
        continue

    print(f"Scraping {title!r} → {url}")
    resp = session.get(url, timeout=10)
    resp.raise_for_status()

    # pandas will find all <table> tags—take the first one
    tables = pd.read_html(resp.text)
    if not tables:
        print(f"No tables found for {title!r}, skipping.")
        continue

    df = tables[0]
    df.insert(0, "Metric", title)
    df_stats.append(df)
    print(f"  Retrieved {len(df)} rows for {title!r}")

    # optional pause to be polite
    time.sleep(random.uniform(1, 2))

if df_stats:
    combined = pd.concat(df_stats, ignore_index=True)
    combined.to_csv(OUTPUT_CSV, index=False)
    print(f"\nDone! {len(combined)} rows saved to {OUTPUT_CSV}")
else:
    print("No data collected.")

Scraping 'Total Offense' → https://stats.ncaa.org/rankings/national_ranking?academic_year=2025.0&division=11.0&ranking_period=84.0&sport_code=MFB&stat_seq=21.0


C:\Users\nelle\AppData\Local\Temp\ipykernel_31148\3436200881.py:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


  Retrieved 135 rows for 'Total Offense'
Scraping 'Rushing Offense' → https://stats.ncaa.org/rankings/national_ranking?academic_year=2025.0&division=11.0&ranking_period=84.0&sport_code=MFB&stat_seq=23.0


C:\Users\nelle\AppData\Local\Temp\ipykernel_31148\3436200881.py:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


  Retrieved 135 rows for 'Rushing Offense'
Scraping 'Passing Offense' → https://stats.ncaa.org/rankings/national_ranking?academic_year=2025.0&division=11.0&ranking_period=84.0&sport_code=MFB&stat_seq=25.0


...

  Retrieved 130 rows for 'Fewest Penalties Per Game'
Scraping 'Fewest Penalty Yards Per Game' → https://stats.ncaa.org/rankings/national_ranking?academic_year=2022.0&division=11.0&ranking_period=70.0&sport_code=MFB&stat_seq=698.0


C:\Users\nelle\AppData\Local\Temp\ipykernel_31148\3436200881.py:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


  Retrieved 130 rows for 'Fewest Penalty Yards Per Game'
Scraping 'Time of Possession' → https://stats.ncaa.org/rankings/national_ranking?academic_year=2022.0&division=11.0&ranking_period=70.0&sport_code=MFB&stat_seq=705.0


C:\Users\nelle\AppData\Local\Temp\ipykernel_31148\3436200881.py:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(resp.text)


  Retrieved 130 rows for 'Time of Possession'

Done! 5310 rows saved to ncaa_all_metrics_raw_data.csv


In [65]:
# Total Offense
df_Total_Offense = df_stats[0].drop(columns = ['Metric', 'Plays','YDS','Yds/Play'])
df_Total_Offense = df_Total_Offense.drop(df_Total_Offense.index [133])
df_Total_Offense = df_Total_Offense.drop(df_Total_Offense.index [133])
df_Total_Offense = df_Total_Offense.rename(columns={'YPG': 'Total Offense YPG'})
df_Total_Offense = df_Total_Offense.rename(columns={'Rank': 'Offensive Rank'})
df_Total_Offense = df_Total_Offense.rename(columns={'Off TDs': 'Offense TDs'})
df_Total_Offense['Season'] = 2024
df_Total_Offense

,Offensive Rank,Team,G,W-L,Offense TDs,Total Offense YPG,Season
0,1,Miami (FL) (ACC),13,10-3,71,537.2,2024
1,2,Ole Miss (SEC),13,10-3,60,526.5,2024
2,3,North Texas (The American),13,6-7,57,488.8,2024
3,4,New Mexico (Mountain West),12,5-7,49,484.3,2024
4,5,Texas St. (Sun Belt),13,8-5,58,476.9,2024
...,...,...,...,...,...,...,...
128,129,Michigan (Big Ten),13,8-5,30,286.2,2024
129,130,Northwestern (Big Ten),12,4-8,19,284.4,2024
130,131,Southern Miss. (Sun Belt),12,1-11,20,283.5,2024
131,132,Florida St. (ACC),12,2-10,19,270.3,2024


In [66]:
# Rushing Offense
df_Rushing_Offense = df_stats[1].drop(columns = ['Metric', 'Rush','Rush Yds','Yds/Rush'])
df_Rushing_Offense = df_Rushing_Offense.drop(df_Rushing_Offense.index[133])
df_Rushing_Offense = df_Rushing_Offense.drop(df_Rushing_Offense.index[133])
df_Rushing_Offense = df_Rushing_Offense.rename(columns={'YPG': 'Rush YPG'})
df_Rushing_Offense = df_Rushing_Offense.rename(columns={'Rank': 'Rushing Rank'})
df_Rushing_Offense = df_Rushing_Offense.rename(columns={'Rush TD': 'Rushing TDs'})
df_Rushing_Offense['Season'] = 2024
df_Rushing_Offense.head(3)

,Rushing Rank,Team,G,W-L,Rushing TDs,Rush YPG,Season
0,1,Army West Point (The American),14,12-2,48,300.5,2024
1,2,New Mexico (Mountain West),12,5-7,37,253.6,2024
2,3,Jacksonville St. (CUSA),14,9-5,51,251.2,2024


In [67]:
df_Passing_Offense = df_stats[2].drop(columns = ['Metric', 'Pass Att', 'Pass Com', 'Pass Yds', 'Yds/Att', 'Yds/Comp'])
df_Passing_Offense = df_Passing_Offense.drop(df_Passing_Offense.index[133])
df_Passing_Offense = df_Passing_Offense.drop(df_Passing_Offense.index[133])
df_Passing_Offense = df_Passing_Offense.rename(columns={'YPG': 'Pass YPG'})
df_Passing_Offense = df_Passing_Offense.rename(columns={'Rank': 'Passing Rank'})
df_Passing_Offense = df_Passing_Offense.rename(columns={'Pass TD': 'Passing TDs'})
df_Passing_Offense['Season'] = 2024
df_Passing_Offense.head(3)

,Passing Rank,Team,G,W-L,Int,Passing TDs,Pass YPG,Season
0,1,Syracuse (ACC),13,10-3,12,34,370.0,2024
1,2,Ole Miss (SEC),13,10-3,7,31,350.8,2024
2,3,Miami (FL) (ACC),13,10-3,8,41,348.2,2024


In [68]:
df_Total_Defense = df_stats[3].drop(columns = ['Metric', 'Plays','YDS','Yds/Play','Off TDs'])
df_Total_Defense = df_Total_Defense.drop(df_Total_Defense.index[133])
df_Total_Defense = df_Total_Defense.drop(df_Total_Defense.index[133])
df_Total_Defense = df_Total_Defense.rename(columns={'YPG': 'Total Defense YPG'})
df_Total_Defense = df_Total_Defense.rename(columns={'Rank': 'Defensive Rank'})
df_Total_Defense['Season'] = 2024
df_Total_Defense.head(3)

,Defensive Rank,Team,G,W-L,Opp TDs,Total Defense YPG,Season
0,1,Ohio St. (Big Ten),16,14-2,24,254.6,2024
1,2,Indiana (Big Ten),13,11-2,24,256.3,2024
2,3,Texas (SEC),16,13-3,27,283.4,2024


In [69]:
df_Rushing_Defense = df_stats[4].drop(columns = ['Metric', 'Opp Rush','Opp Rush Yds','Yds/Rush'])
df_Rushing_Defense = df_Rushing_Defense.drop(df_Rushing_Defense.index[133])
df_Rushing_Defense = df_Rushing_Defense.drop(df_Rushing_Defense.index[133])
df_Rushing_Defense = df_Rushing_Defense.rename(columns={'YPG': 'Opp Rush YPG'})
df_Rushing_Defense = df_Rushing_Defense.rename(columns={'Rank': 'Rushing Defensive Rank'})
df_Rushing_Defense['Season'] = 2024
df_Rushing_Defense.head(3)

,Rushing Defensive Rank,Team,G,W-L,Opp Rush TDs,Opp Rush YPG,Season
0,1,Indiana (Big Ten),13,11-2,12,80.2,2024
1,2,Ole Miss (SEC),13,10-3,7,80.5,2024
2,3,Ohio St. (Big Ten),16,14-2,13,87.6,2024


In [70]:
df_Passing_Defense = df_stats[5].drop(columns = ['Metric', 'Opp Cpl', 'Opp Pass', 'Opp Pass <br/>Yds', 'Yds/Att', 'Yds/Comp'])
df_Passing_Defense = df_Passing_Defense.drop(df_Passing_Defense.index[133])
df_Passing_Defense = df_Passing_Defense.drop(df_Passing_Defense.index[133])
df_Passing_Defense = df_Passing_Defense.rename(columns={'YPG': 'Opp Pass YPG'})
df_Passing_Defense = df_Passing_Defense.rename(columns={'Rank': 'Passing Defensive Rank'})
df_Passing_Defense = df_Passing_Defense.rename(columns={'Opp Pass <br/>TDs': 'Opp Pass TDs'})
df_Passing_Defense['Season'] = 2024
df_Passing_Defense.head(3)

,Passing Defensive Rank,Team,G,W-L,Opp Pass TDs,Opp Pass YPG,Season
0,1,Iowa St. (Big 12),14,11-3,18,165.6,2024
1,2,Washington (Big Ten),13,6-7,16,166.5,2024
2,3,Ohio St. (Big Ten),16,14-2,10,167.0,2024


In [71]:
df_Turnover_Margin = df_stats[6].drop(columns = ['Metric', "Opp Int", "Turn Gain", "Int", "Turn Lost", "Avg"])
df_Turnover_Margin = df_Turnover_Margin.drop(df_Turnover_Margin.index[133])
df_Turnover_Margin = df_Turnover_Margin.drop(df_Turnover_Margin.index[133])
df_Turnover_Margin = df_Turnover_Margin.rename(columns={'Fum Rec': 'Fumbles Recovered'})
df_Turnover_Margin = df_Turnover_Margin.rename(columns={'Fum Lost': 'Fumbles Lost'})
df_Turnover_Margin = df_Turnover_Margin.rename(columns={'Margin': 'Turnover Margin'})
df_Turnover_Margin = df_Turnover_Margin.rename(columns={'Rank': 'TM Rank'})
df_Turnover_Margin['Season'] = 2024
df_Turnover_Margin

,TM Rank,Team,G,W-L,Fumbles Recovered,Fumbles Lost,Turnover Margin,Season
0,1,James Madison (Sun Belt),13,9-4,12,5,20,2024
1,2,Memphis (The American),13,11-2,15,2,18,2024
2,3,Indiana (Big Ten),13,11-2,9,3,15,2024
3,4,Clemson (ACC),14,10-4,10,3,16,2024
4,5,Notre Dame (FBS Independent),16,14-2,14,7,18,2024
...,...,...,...,...,...,...,...,...
128,129,New Mexico St. (CUSA),12,3-9,1,11,-13,2024
129,130,Purdue (Big Ten),12,1-11,1,10,-14,2024
130,131,Florida St. (ACC),12,2-10,2,9,-16,2024
131,132,Central Mich. (MAC),12,4-8,3,10,-17,2024


In [72]:
df_Penalties = df_stats[7].drop(columns = ["Metric"])
df_Penalties = df_Penalties.drop(df_Penalties.index[133])
df_Penalties = df_Penalties.drop(df_Penalties.index[133])
df_Penalties = df_Penalties.rename(columns={'PenPerGame': 'Penalties Per Game'})
df_Penalties = df_Penalties.rename(columns={'Rank': 'PPG Rank'})
df_Penalties['Season'] = 2024
df_Penalties.head(3)

,PPG Rank,Team,G,W-L,Penalties,Penalties Per Game,Season
0,1,Iowa (Big Ten),13,8-5,41,3.15,2024
1,2,Air Force (Mountain West),12,5-7,38,3.17,2024
2,3,Army West Point (The American),14,12-2,45,3.21,2024


In [73]:
df_Penalties_YPG = df_stats[8].drop(columns = ["Metric", "Penalties", "PYards"])
df_Penalties_YPG = df_Penalties_YPG.drop(df_Penalties_YPG.index[133])
df_Penalties_YPG = df_Penalties_YPG.drop(df_Penalties_YPG.index[133])
df_Penalties_YPG = df_Penalties_YPG.rename(columns={'YPG': 'Penalty YPG'})
df_Penalties_YPG = df_Penalties_YPG.rename(columns={'Rank': 'Penalty YPG Rank'})
df_Penalties_YPG['Season'] = 2024
df_Penalties_YPG.head(3)

,Penalty YPG Rank,Team,G,W-L,Penalty YPG,Season
0,1,Iowa St. (Big 12),14,11-3,28.00,2024
1,2,Air Force (Mountain West),12,5-7,28.83,2024
2,3,Iowa (Big Ten),13,8-5,29.85,2024


In [74]:
# Time of Possession
df_TOP = df_stats[9].drop(columns = ["Metric", "TOP"])
df_TOP = df_TOP.drop(df_TOP.index[133])
df_TOP = df_TOP.drop(df_TOP.index[133])
df_TOP = df_TOP.rename(columns={'AvgTOP': 'Avg Time of Possession'})
df_TOP = df_TOP.rename(columns={'Rank': 'TOP Rank'})
df_TOP['Season'] = 2024
df_TOP.head(3)

,TOP Rank,Team,G,W-L,Avg Time of Possession,Season
0,1,Army West Point (The American),14,12-2,34:39,2024
1,2,Air Force (Mountain West),12,5-7,33:50,2024
2,3,NIU (MAC),13,8-5,33:21,2024


In [75]:
# Total Offense
df_Total_Offense23 = df_stats[10].drop(columns = ['Metric', 'Plays','YDS','Yds/Play'])
df_Total_Offense23 = df_Total_Offense23.drop(df_Total_Offense23.index [130])
df_Total_Offense23 = df_Total_Offense23.drop(df_Total_Offense23.index [130])
df_Total_Offense23 = df_Total_Offense23.drop(df_Total_Offense23.index [130])
df_Total_Offense23 = df_Total_Offense23.drop(df_Total_Offense23.index [130])
df_Total_Offense23 = df_Total_Offense23.rename(columns={'YPG': 'Total Offense YPG'})
df_Total_Offense23 = df_Total_Offense23.rename(columns={'Rank': 'Offensive Rank'})
df_Total_Offense23 = df_Total_Offense23.rename(columns={'Off TDs': 'Offense TDs'})
df_Total_Offense23['Season'] = 2023
df_Total_Offense23

,Offensive Rank,Team,G,W-L,Offense TDs,Total Offense YPG,Season
0,1,LSU (SEC),13,10-3,78,543.5,2023
1,2,Oregon (Pac-12),14,12-2,82,531.4,2023
2,3,Oklahoma (Big 12),13,10-3,66,507.0,2023
3,4,Liberty (CUSA),14,13-1,71,499.1,2023
4,5,Georgia (SEC),14,13-1,69,496.5,2023
...,...,...,...,...,...,...,...
125,126,Akron (MAC),12,2-10,22,278.6,2023
126,127,East Carolina (AAC),12,2-10,19,277.3,2023
127,128,Kent St. (MAC),12,1-11,19,270.4,2023
128,129,Eastern Mich. (MAC),13,6-7,28,262.8,2023


In [76]:
# Rushing Offense
df_Rushing_Offense23 = df_stats[11].drop(columns = ['Metric', 'Rush','Rush Yds','Yds/Rush'])
df_Rushing_Offense23 = df_Rushing_Offense23.drop(df_Rushing_Offense23.index[130])
df_Rushing_Offense23 = df_Rushing_Offense23.drop(df_Rushing_Offense23.index[130])
df_Rushing_Offense23 = df_Rushing_Offense23.drop(df_Rushing_Offense23.index[130])
df_Rushing_Offense23 = df_Rushing_Offense23.drop(df_Rushing_Offense23.index[130])
df_Rushing_Offense23 = df_Rushing_Offense23.rename(columns={'YPG': 'Rush YPG'})
df_Rushing_Offense23 = df_Rushing_Offense23.rename(columns={'Rank': 'Rushing Rank'})
df_Rushing_Offense23 = df_Rushing_Offense23.rename(columns={'Rush TD': 'Rushing TDs'})
df_Rushing_Offense23['Season'] = 2023
df_Rushing_Offense23

,Rushing Rank,Team,G,W-L,Rushing TDs,Rush YPG,Season
0,1,Liberty (CUSA),14,13-1,39,293.3,2023
1,2,Air Force (Mountain West),13,9-4,36,281.5,2023
2,3,West Virginia (Big 12),13,9-4,32,228.9,2023
3,4,UCF (Big 12),13,6-7,28,228.2,2023
4,5,Cincinnati (Big 12),12,3-9,14,217.1,2023
...,...,...,...,...,...,...,...
125,126,Washington St. (Pac-12),12,5-7,17,84.8,2023
126,127,South Carolina (SEC),12,5-7,17,84.4,2023
127,128,Akron (MAC),12,2-10,12,81.0,2023
128,129,Hawaii (Mountain West),13,5-8,6,76.1,2023


In [77]:
df_Passing_Offense23 = df_stats[12].drop(columns=['Metric', 'Pass Att', 'Pass Com', 'Pass Yds', 'Yds/Att', 'Yds/Comp'])
df_Passing_Offense23 = df_Passing_Offense23.drop(df_Passing_Offense23.index[130])
df_Passing_Offense23 = df_Passing_Offense23.drop(df_Passing_Offense23.index[130])
df_Passing_Offense23 = df_Passing_Offense23.drop(df_Passing_Offense23.index[130])
df_Passing_Offense23 = df_Passing_Offense23.drop(df_Passing_Offense23.index[130])
df_Passing_Offense23 = df_Passing_Offense23.rename(columns={'YPG': 'Pass YPG'})
df_Passing_Offense23 = df_Passing_Offense23.rename(columns={'Rank': 'Passing Rank'})
df_Passing_Offense23 = df_Passing_Offense23.rename(columns={'Pass TD': 'Passing TDs'})
df_Passing_Offense23['Season'] = 2023
df_Passing_Offense23

,Passing Rank,Team,G,W-L,Int,Passing TDs,Pass YPG,Season
0,1,Oregon (Pac-12),14,12-2,5,49,346.9,2023
1,2,Washington (Pac-12),15,14-1,12,38,343.7,2023
2,3,LSU (SEC),13,10-3,5,44,338.9,2023
3,4,Washington St. (Pac-12),12,5-7,9,27,336.8,2023
4,5,Southern California (Pac-12),13,8-5,6,37,333.0,2023
...,...,...,...,...,...,...,...,...
125,126,Nebraska (Big Ten),12,5-7,16,10,135.9,2023
126,127,Iowa (Big Ten),14,10-4,11,9,118.6,2023
127,128,Navy (AAC),12,5-7,5,10,105.7,2023
128,129,Army West Point (FBS Independent),12,6-6,11,7,102.8,2023


In [78]:
# Total Defense for 2023
df_Total_Defense23 = df_stats[13].drop(columns=['Metric', 'Plays', 'YDS', 'Yds/Play', 'Off TDs'])
df_Total_Defense23 = df_Total_Defense23.drop(df_Total_Defense23.index[130])
df_Total_Defense23 = df_Total_Defense23.drop(df_Total_Defense23.index[130])
df_Total_Defense23 = df_Total_Defense23.drop(df_Total_Defense23.index[130])
df_Total_Defense23 = df_Total_Defense23.drop(df_Total_Defense23.index[130])
df_Total_Defense23 = df_Total_Defense23.rename(columns={'YPG': 'Total Defense YPG'})
df_Total_Defense23 = df_Total_Defense23.rename(columns={'Rank': 'Defensive Rank'})
df_Total_Defense23['Season'] = 2023
df_Total_Defense23

,Defensive Rank,Team,G,W-L,Opp TDs,Total Defense YPG,Season
0,1,Michigan (Big Ten),15,15-0,17,247.0,2023
1,2,Penn St. (Big Ten),13,10-3,20,247.6,2023
2,3,Ohio St. (Big Ten),13,11-2,15,265.4,2023
3,4,Ohio (MAC),13,10-3,24,273.9,2023
4,5,Notre Dame (FBS Independent),13,10-3,23,276.3,2023
...,...,...,...,...,...,...,...
125,126,ULM (Sun Belt),12,2-10,52,445.8,2023
126,127,Colorado (Pac-12),12,4-8,56,453.3,2023
127,128,Vanderbilt (SEC),12,2-10,54,454.9,2023
128,129,Stanford (Pac-12),12,3-9,63,461.7,2023


In [79]:
# Rushing Defense for 2023
df_Rushing_Defense23 = df_stats[14].drop(columns=['Metric', 'Opp Rush', 'Opp Rush Yds', 'Yds/Rush'])
df_Rushing_Defense23 = df_Rushing_Defense23.drop(df_Rushing_Defense23.index[130])
df_Rushing_Defense23 = df_Rushing_Defense23.drop(df_Rushing_Defense23.index[130])
df_Rushing_Defense23 = df_Rushing_Defense23.drop(df_Rushing_Defense23.index[130])
df_Rushing_Defense23 = df_Rushing_Defense23.drop(df_Rushing_Defense23.index[130])
df_Rushing_Defense23 = df_Rushing_Defense23.rename(columns={'YPG': 'Opp Rush YPG'})
df_Rushing_Defense23 = df_Rushing_Defense23.rename(columns={'Rank': 'Rushing Defensive Rank'})
df_Rushing_Defense23['Season'] = 2023
df_Rushing_Defense23

,Rushing Defensive Rank,Team,G,W-L,Opp Rush TDs,Opp Rush YPG,Season
0,1,Penn St. (Big Ten),13,10-3,10,75.5,2023
1,2,UCLA (Pac-12),13,8-5,5,80.8,2023
2,3,Texas (Big 12),14,12-2,9,82.4,2023
3,4,Utah (Pac-12),13,8-5,13,82.8,2023
4,5,Michigan (Big Ten),15,15-0,9,90.0,2023
...,...,...,...,...,...,...,...
125,126,Utah St. (Mountain West),13,6-7,31,213.5,2023
126,127,Massachusetts (FBS Independent),12,3-9,30,220.8,2023
127,128,Georgia Tech (ACC),13,7-6,28,221.3,2023
128,129,Louisiana Tech (CUSA),12,3-9,32,232.8,2023


In [80]:
# Passing Defense for 2023
df_Passing_Defense23 = df_stats[15].drop(columns=['Metric', 'Opp Cpl', 'Opp Pass', 'Opp Pass <br/>Yds', 'Yds/Att', 'Yds/Comp'])
df_Passing_Defense23 = df_Passing_Defense23.drop(df_Passing_Defense23.index[130])
df_Passing_Defense23 = df_Passing_Defense23.drop(df_Passing_Defense23.index[130])
df_Passing_Defense23 = df_Passing_Defense23.drop(df_Passing_Defense23.index[130])
df_Passing_Defense23 = df_Passing_Defense23.drop(df_Passing_Defense23.index[130])
df_Passing_Defense23 = df_Passing_Defense23.rename(columns={'YPG': 'Opp Pass YPG'})
df_Passing_Defense23 = df_Passing_Defense23.rename(columns={'Rank': 'Passing Defensive Rank'})
df_Passing_Defense23 = df_Passing_Defense23.rename(columns={'Opp Pass <br/>TDs': 'Opp Pass TDs'})
df_Passing_Defense23['Season'] = 2023
df_Passing_Defense23

,Passing Defensive Rank,Team,G,W-L,Opp Pass TDs,Opp Pass YPG,Season
0,1,Ohio St. (Big Ten),13,11-2,9,145.9,2023
1,2,Michigan (Big Ten),15,15-0,8,157.0,2023
2,3,Notre Dame (FBS Independent),13,10-3,8,157.3,2023
3,4,Virginia Tech (ACC),13,7-6,16,168.8,2023
4,5,Iowa (Big Ten),14,10-4,11,170.7,2023
...,...,...,...,...,...,...,...
125,126,Vanderbilt (SEC),12,2-10,26,279.6,2023
126,127,Tulsa (AAC),12,4-8,24,279.8,2023
127,128,California (Pac-12),13,6-7,30,280.7,2023
128,129,South Fla. (AAC),13,7-6,29,288.8,2023


In [81]:
# Turnover Margin for 2023
df_Turnover_Margin23 = df_stats[16].drop(columns=['Metric', "Opp Int", "Turn Gain", "Int", "Turn Lost", "Avg"])
df_Turnover_Margin23 = df_Turnover_Margin23.drop(df_Turnover_Margin23.index[130])
df_Turnover_Margin23 = df_Turnover_Margin23.drop(df_Turnover_Margin23.index[130])
df_Turnover_Margin23 = df_Turnover_Margin23.drop(df_Turnover_Margin23.index[130])
df_Turnover_Margin23 = df_Turnover_Margin23.drop(df_Turnover_Margin23.index[130])
df_Turnover_Margin23 = df_Turnover_Margin23.rename(columns={'Fum Rec': 'Fumbles Recovered'})
df_Turnover_Margin23 = df_Turnover_Margin23.rename(columns={'Fum Lost': 'Fumbles Lost'})
df_Turnover_Margin23 = df_Turnover_Margin23.rename(columns={'Margin': 'Turnover Margin'})
df_Turnover_Margin23 = df_Turnover_Margin23.rename(columns={'Rank': 'TM Rank'})
df_Turnover_Margin23['Season'] = 2023
df_Turnover_Margin23

,TM Rank,Team,G,W-L,Fumbles Recovered,Fumbles Lost,Turnover Margin,Season
0,1,Michigan (Big Ten),15,15-0,9,3,19,2023
1,2,Penn St. (Big Ten),13,10-3,12,6,16,2023
2,3,Northwestern (Big Ten),13,8-5,9,1,13,2023
3,4,Navy (AAC),12,5-7,14,8,11,2023
4,5,Kansas St. (Big 12),13,9-4,10,2,11,2023
...,...,...,...,...,...,...,...,...
125,126,Marshall (Sun Belt),13,6-7,5,11,-11,2023
126,127,Arizona St. (Pac-12),12,3-9,4,6,-11,2023
127,128,Rice (AAC),13,6-7,7,10,-14,2023
128,129,Nebraska (Big Ten),12,5-7,5,15,-17,2023


In [82]:
# Fewest Penalties Per Game for 2023
df_Penalties23 = df_stats[17].drop(columns=["Metric"])
df_Penalties23 = df_Penalties23.drop(df_Penalties23.index[130])
df_Penalties23 = df_Penalties23.rename(columns={'PenPerGame': 'Penalties Per Game'})
df_Penalties23 = df_Penalties23.rename(columns={'Rank': 'PPG Rank'})
df_Penalties23['Season'] = 2023
df_Penalties23

,PPG Rank,Team,G,W-L,Penalties,Penalties Per Game,Season
0,1,Air Force (Mountain West),13,9-4,39,3.00,2023
1,1,Michigan (Big Ten),15,15-0,45,3.00,2023
2,1,Minnesota (Big Ten),13,6-7,39,3.00,2023
3,4,Fla. Atlantic (AAC),12,4-8,41,3.42,2023
4,5,Iowa (Big Ten),14,10-4,51,3.64,2023
...,...,...,...,...,...,...,...
128,129,Colorado (Pac-12),12,4-8,107,8.92,2023
129,130,New Mexico (Mountain West),12,4-8,119,9.92,2023
131,-,Jacksonville St. (CUSA),13,9-4,63,4.85,2023
132,-,Sam Houston (CUSA),12,3-9,61,5.08,2023


In [83]:
# Fewest Penalty Yards Per Game for 2023
df_Penalties_YPG23 = df_stats[18].drop(columns=["Metric", "Penalties", "PYards"])
df_Penalties_YPG23 = df_Penalties_YPG23.drop(df_Penalties_YPG23.index[130])
df_Penalties_YPG23 = df_Penalties_YPG23.drop(df_Penalties_YPG23.index[130])
df_Penalties_YPG23 = df_Penalties_YPG23.drop(df_Penalties_YPG23.index[130])
df_Penalties_YPG23 = df_Penalties_YPG23.drop(df_Penalties_YPG23.index[130])
df_Penalties_YPG23 = df_Penalties_YPG23.rename(columns={'YPG': 'Penalty YPG'})
df_Penalties_YPG23 = df_Penalties_YPG23.rename(columns={'Rank': 'Penalty YPG Rank'})
df_Penalties_YPG23['Season'] = 2023
df_Penalties_YPG23

,Penalty YPG Rank,Team,G,W-L,Penalty YPG,Season
0,1,Air Force (Mountain West),13,9-4,24.77,2023
1,2,Minnesota (Big Ten),13,6-7,27.08,2023
2,3,Michigan (Big Ten),15,15-0,27.47,2023
3,4,Baylor (Big 12),12,3-9,28.92,2023
4,5,Iowa (Big Ten),14,10-4,30.93,2023
...,...,...,...,...,...,...
125,126,Colorado St. (Mountain West),12,5-7,69.83,2023
126,127,Utah St. (Mountain West),13,6-7,70.85,2023
127,128,Massachusetts (FBS Independent),12,3-9,70.92,2023
128,129,Colorado (Pac-12),12,4-8,71.17,2023


In [84]:
# Time of Possession for 2023
df_TOP23 = df_stats[19].drop(columns=["Metric", "TOP"])
df_TOP23 = df_TOP23.drop(df_TOP23.index[130])
df_TOP23 = df_TOP23.drop(df_TOP23.index[130])
df_TOP23 = df_TOP23.drop(df_TOP23.index[130])
df_TOP23 = df_TOP23.drop(df_TOP23.index[130])
df_TOP23 = df_TOP23.rename(columns={'AvgTOP': 'Avg Time of Possession'})
df_TOP23 = df_TOP23.rename(columns={'Rank': 'TOP Rank'})
df_TOP23['Season'] = 2023
df_TOP23

,TOP Rank,Team,G,W-L,Avg Time of Possession,Season
0,1,Utah (Pac-12),13,8-5,34:04,2023
1,2,Georgia (SEC),14,13-1,33:28,2023
2,3,Air Force (Mountain West),13,9-4,33:27,2023
3,4,Ohio (MAC),13,10-3,33:25,2023
4,5,Clemson (ACC),13,9-4,33:19,2023
...,...,...,...,...,...,...
125,126,FIU (CUSA),12,4-8,27:05,2023
126,127,Utah St. (Mountain West),13,6-7,26:51,2023
127,128,Arkansas St. (Sun Belt),13,6-7,26:17,2023
128,129,Old Dominion (Sun Belt),13,6-7,24:49,2023


In [85]:
# Total Offense for 2022
df_Total_Offense22 = df_stats[20].drop(columns=['Metric', 'Plays', 'YDS', 'Yds/Play'])
df_Total_Offense22 = df_Total_Offense22.drop(df_Total_Offense22.index[130])
df_Total_Offense22 = df_Total_Offense22.drop(df_Total_Offense22.index[130])
df_Total_Offense22 = df_Total_Offense22.rename(columns={'YPG': 'Total Offense YPG'})
df_Total_Offense22 = df_Total_Offense22.rename(columns={'Rank': 'Offensive Rank'})
df_Total_Offense22 = df_Total_Offense22.rename(columns={'Off TDs': 'Offense TDs'})
df_Total_Offense22['Season'] = 2022
df_Total_Offense22

,Offensive Rank,Team,G,W-L,Offense TDs,Total Offense YPG,Season
0,1,Tennessee (SEC),13,11-2,78,525.5,2022
1,2,Washington (Pac-12),13,11-2,65,515.8,2022
2,3,Southern California (Pac-12),14,11-3,73,506.6,2022
3,4,UCLA (Pac-12),13,9-4,65,503.6,2022
4,5,Georgia (SEC),15,15-0,75,501.2,2022
...,...,...,...,...,...,...,...
125,126,Rutgers (Big Ten),12,4-8,22,282.1,2022
126,127,Colorado (Pac-12),12,1-11,21,281.3,2022
127,128,Massachusetts (FBS Independent),12,1-11,14,265.8,2022
128,129,Iowa (Big Ten),13,8-5,19,251.5,2022


In [86]:
# Rushing Offense for 2022
df_Rushing_Offense22 = df_stats[21].drop(columns=['Metric', 'Rush', 'Rush Yds', 'Yds/Rush'])
df_Rushing_Offense22 = df_Rushing_Offense22.drop(df_Rushing_Offense22.index[130])
df_Rushing_Offense22 = df_Rushing_Offense22.drop(df_Rushing_Offense22.index[130])
df_Rushing_Offense22 = df_Rushing_Offense22.rename(columns={'YPG': 'Rush YPG'})
df_Rushing_Offense22 = df_Rushing_Offense22.rename(columns={'Rank': 'Rushing Rank'})
df_Rushing_Offense22 = df_Rushing_Offense22.rename(columns={'Rush TD': 'Rushing TDs'})
df_Rushing_Offense22['Season'] = 2022
df_Rushing_Offense22

,Rushing Rank,Team,G,W-L,Rushing TDs,Rush YPG,Season
0,1,Air Force (Mountain West),13,10-3,37,326.7,2022
1,2,Army West Point (FBS Independent),12,6-6,38,289.4,2022
2,3,Ole Miss (SEC),13,8-5,33,256.6,2022
3,4,Navy (The American),12,4-8,22,241.2,2022
4,5,Michigan (Big Ten),14,13-1,41,238.9,2022
...,...,...,...,...,...,...,...
125,126,Colorado St. (Mountain West),12,3-9,4,89.0,2022
126,127,Arkansas St. (Sun Belt),12,3-9,16,88.2,2022
127,128,Temple (The American),12,3-9,11,82.4,2022
128,129,Mississippi St. (SEC),13,9-4,12,81.7,2022


In [87]:
# Passing Offense for 2022
df_Passing_Offense22 = df_stats[22].drop(columns=['Metric', 'Pass Att', 'Pass Com', 'Pass Yds', 'Yds/Att', 'Yds/Comp'])
df_Passing_Offense22 = df_Passing_Offense22.drop(df_Passing_Offense22.index[130])
df_Passing_Offense22 = df_Passing_Offense22.drop(df_Passing_Offense22.index[130])
df_Passing_Offense22 = df_Passing_Offense22.rename(columns={'YPG': 'Pass YPG'})
df_Passing_Offense22 = df_Passing_Offense22.rename(columns={'Rank': 'Passing Rank'})
df_Passing_Offense22 = df_Passing_Offense22.rename(columns={'Pass TD': 'Passing TDs'})
df_Passing_Offense22['Season'] = 2022
df_Passing_Offense22

,Passing Rank,Team,G,W-L,Int,Passing TDs,Pass YPG,Season
0,1,Washington (Pac-12),13,11-2,9,32,369.8,2022
1,2,Western Ky. (CUSA),14,9-5,12,43,352.2,2022
2,3,Southern California (Pac-12),14,11-3,5,43,335.4,2022
3,4,Ga. Southern (Sun Belt),13,6-7,16,28,329.9,2022
4,5,Tennessee (SEC),13,11-2,3,38,326.1,2022
...,...,...,...,...,...,...,...,...
125,126,UConn (FBS Independent),13,6-7,10,10,111.9,2022
126,127,New Mexico (Mountain West),12,2-10,9,3,104.9,2022
127,128,Navy (The American),12,4-8,6,9,85.7,2022
128,129,Army West Point (FBS Independent),12,6-6,4,5,76.7,2022


In [88]:
# Total Defense for 2022
df_Total_Defense22 = df_stats[23].drop(columns=['Metric', 'Plays', 'YDS', 'Yds/Play', 'Off TDs'])
df_Total_Defense22 = df_Total_Defense22.drop(df_Total_Defense22.index[130])
df_Total_Defense22 = df_Total_Defense22.drop(df_Total_Defense22.index[130])
df_Total_Defense22 = df_Total_Defense22.rename(columns={'YPG': 'Total Defense YPG'})
df_Total_Defense22 = df_Total_Defense22.rename(columns={'Rank': 'Defensive Rank'})
df_Total_Defense22['Season'] = 2022
df_Total_Defense22

,Defensive Rank,Team,G,W-L,Opp TDs,Total Defense YPG,Season
0,1,Air Force (Mountain West),13,10-3,20,254.4,2022
1,2,Iowa (Big Ten),13,8-5,18,270.8,2022
2,3,Illinois (Big Ten),13,8-5,16,273.5,2022
3,4,Iowa St. (Big 12),12,4-8,30,285.6,2022
4,5,Michigan (Big Ten),14,13-1,25,292.1,2022
...,...,...,...,...,...,...,...
125,126,Kansas (Big 12),13,6-7,58,469.2,2022
126,127,Charlotte (CUSA),12,3-9,62,483.8,2022
127,128,Ga. Southern (Sun Belt),13,6-7,49,487.8,2022
128,129,Colorado (Pac-12),12,1-11,72,509.8,2022


In [89]:
# Rushing Defense for 2022
df_Rushing_Defense22 = df_stats[24].drop(columns=['Metric', 'Opp Rush', 'Opp Rush Yds', 'Yds/Rush'])
df_Rushing_Defense22 = df_Rushing_Defense22.drop(df_Rushing_Defense22.index[130])
df_Rushing_Defense22 = df_Rushing_Defense22.drop(df_Rushing_Defense22.index[130])
df_Rushing_Defense22 = df_Rushing_Defense22.rename(columns={'YPG': 'Opp Rush YPG'})
df_Rushing_Defense22 = df_Rushing_Defense22.rename(columns={'Rank': 'Rushing Defensive Rank'})
df_Rushing_Defense22['Season'] = 2022
df_Rushing_Defense22

,Rushing Defensive Rank,Team,G,W-L,Opp Rush TDs,Opp Rush YPG,Season
0,1,Georgia (SEC),15,15-0,7,77.1,2022
1,2,Navy (The American),12,4-8,7,88.9,2022
2,3,Marshall (Sun Belt),13,9-4,10,93.0,2022
3,4,South Alabama (Sun Belt),13,10-3,8,93.9,2022
4,5,Air Force (Mountain West),13,10-3,10,95.3,2022
...,...,...,...,...,...,...,...
125,126,Stanford (Pac-12),12,3-9,25,224.4,2022
126,127,Ga. Southern (Sun Belt),13,6-7,29,230.9,2022
127,128,South Fla. (The American),12,1-11,37,234.6,2022
128,129,Louisiana Tech (CUSA),12,3-9,34,243.2,2022


In [90]:
# Passing Defense for 2022
df_Passing_Defense22 = df_stats[25].drop(columns=['Metric', 'Opp Cpl', 'Opp Pass', 'Opp Pass <br/>Yds', 'Yds/Att', 'Yds/Comp'])
df_Passing_Defense22 = df_Passing_Defense22.drop(df_Passing_Defense22.index[130])
df_Passing_Defense22 = df_Passing_Defense22.drop(df_Passing_Defense22.index[130])
df_Passing_Defense22 = df_Passing_Defense22.rename(columns={'YPG': 'Opp Pass YPG'})
df_Passing_Defense22 = df_Passing_Defense22.rename(columns={'Rank': 'Passing Defensive Rank'})
df_Passing_Defense22 = df_Passing_Defense22.rename(columns={'Opp Pass <br/>TDs': 'Opp Pass TDs'})
df_Passing_Defense22['Season'] = 2022
df_Passing_Defense22

,Passing Defensive Rank,Team,G,W-L,Opp Pass TDs,Opp Pass YPG,Season
0,1,Texas A&M (SEC),12,5-7,15,156.2,2022
1,2,Air Force (Mountain West),13,10-3,10,159.1,2022
2,3,Army West Point (FBS Independent),12,6-6,15,160.0,2022
3,4,Florida St. (ACC),13,10-3,18,165.4,2022
4,5,Boise St. (Mountain West),14,10-4,13,167.7,2022
...,...,...,...,...,...,...,...
125,126,Tennessee (SEC),13,11-2,21,289.5,2022
126,127,Vanderbilt (SEC),12,5-7,29,290.5,2022
127,128,East Carolina (The American),13,8-5,27,291.8,2022
128,129,Middle Tenn. (CUSA),13,8-5,29,292.7,2022


In [91]:
# Turnover Margin for 2022
df_Turnover_Margin22 = df_stats[26].drop(columns=['Metric', "Opp Int", "Turn Gain", "Int", "Turn Lost", "Avg"])
df_Turnover_Margin22 = df_Turnover_Margin22.drop(df_Turnover_Margin22.index[130])
df_Turnover_Margin22 = df_Turnover_Margin22.drop(df_Turnover_Margin22.index[130])
df_Turnover_Margin22 = df_Turnover_Margin22.rename(columns={'Fum Rec': 'Fumbles Recovered'})
df_Turnover_Margin22 = df_Turnover_Margin22.rename(columns={'Fum Lost': 'Fumbles Lost'})
df_Turnover_Margin22 = df_Turnover_Margin22.rename(columns={'Margin': 'Turnover Margin'})
df_Turnover_Margin22 = df_Turnover_Margin22.rename(columns={'Rank': 'TM Rank'})
df_Turnover_Margin22['Season'] = 2022
df_Turnover_Margin22

,TM Rank,Team,G,W-L,Fumbles Recovered,Fumbles Lost,Turnover Margin,Season
0,1,Southern California (Pac-12),14,11-3,10,2,22,2022
1,2,Duke (ACC),13,9-4,16,4,16,2022
2,3,Illinois (Big Ten),13,8-5,8,12,15,2022
3,4,East Carolina (The American),13,8-5,9,2,13,2022
4,5,Middle Tenn. (CUSA),13,8-5,10,7,12,2022
...,...,...,...,...,...,...,...,...
125,124,Temple (The American),12,3-9,6,11,-11,2022
126,127,Rice (CUSA),13,5-8,11,12,-13,2022
127,128,Akron (MAC),12,2-10,4,14,-13,2022
128,129,Central Mich. (MAC),12,4-8,6,18,-18,2022


In [92]:
# Fewest Penalties Per Game for 2022
df_Penalties22 = df_stats[27].drop(columns=["Metric"])
df_Penalties22 = df_Penalties22.drop(df_Penalties22.index[130])
df_Penalties22 = df_Penalties22.drop(df_Penalties22.index[130])
df_Penalties22 = df_Penalties22.rename(columns={'PenPerGame': 'Penalties Per Game'})
df_Penalties22 = df_Penalties22.rename(columns={'Rank': 'PPG Rank'})
df_Penalties22['Season'] = 2022
df_Penalties22

,PPG Rank,Team,G,W-L,Penalties,Penalties Per Game,Season
0,1,Minnesota (Big Ten),13,9-4,46,3.54,2022
1,1,Oklahoma St. (Big 12),13,7-6,46,3.54,2022
2,3,Navy (The American),12,4-8,46,3.83,2022
3,4,Air Force (Mountain West),13,10-3,51,3.92,2022
4,5,Tulane (The American),14,12-2,56,4.00,2022
...,...,...,...,...,...,...,...
125,125,Eastern Mich. (MAC),13,9-4,103,7.92,2022
126,125,Houston (The American),13,8-5,103,7.92,2022
127,128,Tennessee (SEC),13,11-2,105,8.08,2022
128,129,Syracuse (ACC),13,7-6,111,8.54,2022


In [93]:
# Fewest Penalty Yards Per Game for 2022
df_Penalties_YPG22 = df_stats[28].drop(columns=["Metric", "Penalties", "PYards"])
df_Penalties_YPG22 = df_Penalties_YPG22.drop(df_Penalties_YPG22.index[130])
df_Penalties_YPG22 = df_Penalties_YPG22.drop(df_Penalties_YPG22.index[130])
df_Penalties_YPG22 = df_Penalties_YPG22.rename(columns={'YPG': 'Penalty YPG'})
df_Penalties_YPG22 = df_Penalties_YPG22.rename(columns={'Rank': 'Penalty YPG Rank'})
df_Penalties_YPG22['Season'] = 2022
df_Penalties_YPG22

,Penalty YPG Rank,Team,G,W-L,Penalty YPG,Season
0,1,Air Force (Mountain West),13,10-3,29.69,2022
1,2,Navy (The American),12,4-8,31.33,2022
2,3,Minnesota (Big Ten),13,9-4,31.77,2022
3,4,Michigan (Big Ten),14,13-1,32.00,2022
4,5,Iowa (Big Ten),13,8-5,32.69,2022
...,...,...,...,...,...,...
125,126,Maryland (Big Ten),13,8-5,71.08,2022
126,127,South Alabama (Sun Belt),13,10-3,71.92,2022
127,128,Arizona St. (Pac-12),12,3-9,72.33,2022
128,129,Eastern Mich. (MAC),13,9-4,73.46,2022


In [94]:
# Time of Possession for 2022
df_TOP22 = df_stats[29].drop(columns=["Metric", "TOP"])
df_TOP22 = df_TOP22.drop(df_TOP22.index[130])
df_TOP22 = df_TOP22.drop(df_TOP22.index[130])
df_TOP22 = df_TOP22.rename(columns={'AvgTOP': 'Avg Time of Possession'})
df_TOP22 = df_TOP22.rename(columns={'Rank': 'TOP Rank'})
df_TOP22['Season'] = 2022
df_TOP22

,TOP Rank,Team,G,W-L,Avg Time of Possession,Season
0,1,Air Force (Mountain West),13,10-3,36:33,2022
1,2,Navy (The American),12,4-8,34:38,2022
2,3,Minnesota (Big Ten),13,9-4,34:31,2022
3,4,Michigan (Big Ten),14,13-1,33:48,2022
4,5,UTEP (CUSA),12,5-7,33:47,2022
...,...,...,...,...,...,...
125,126,Oklahoma (Big 12),13,6-7,26:23,2022
126,127,Michigan St. (Big Ten),12,5-7,26:13,2022
127,128,Indiana (Big Ten),12,4-8,25:31,2022
128,129,Old Dominion (Sun Belt),12,3-9,25:12,2022


In [95]:
df_Total_Offense21 = df_stats[30].drop(columns=['Metric', 'Plays', 'YDS', 'Yds/Play'])
df_Total_Offense21 = df_Total_Offense21.rename(columns={'YPG': 'Total Offense YPG'})
df_Total_Offense21 = df_Total_Offense21.rename(columns={'Rank': 'Offensive Rank'})
df_Total_Offense21 = df_Total_Offense21.rename(columns={'Off TDs': 'Offense TDs'})
df_Total_Offense21['Season'] = 2021
df_Total_Offense21

,Offensive Rank,Team,G,W-L,Offense TDs,Total Offense YPG,Season
0,1,Ohio St. (Big Ten),13,11-2,70,561.5,2021
1,2,Western Ky. (CUSA),14,9-5,76,535.3,2021
2,3,Virginia (ACC),12,6-6,52,515.8,2021
3,4,Kent St. (MAC),14,7-7,57,495.2,2021
4,5,Coastal Carolina (Sun Belt),13,11-2,69,494.7,2021
...,...,...,...,...,...,...,...
125,126,Navy (AAC),12,4-8,25,282.3,2021
126,127,Southern Miss. (CUSA),12,3-9,22,266.3,2021
127,128,UConn (FBS Independent),12,1-11,19,265.0,2021
128,129,Colorado (Pac-12),12,4-8,22,257.4,2021


In [96]:
# Rushing Offense for 2021
df_Rushing_Offense21 = df_stats[31].drop(columns=['Metric', 'Rush', 'Rush Yds', 'Yds/Rush'])
df_Rushing_Offense21 = df_Rushing_Offense21.rename(columns={'YPG': 'Rush YPG'})
df_Rushing_Offense21 = df_Rushing_Offense21.rename(columns={'Rank': 'Rushing Rank'})
df_Rushing_Offense21 = df_Rushing_Offense21.rename(columns={'Rush TD': 'Rushing TDs'})
df_Rushing_Offense21['Season'] = 2021
df_Rushing_Offense21

,Rushing Rank,Team,G,W-L,Rushing TDs,Rush YPG,Season
0,1,Air Force (Mountain West),13,10-3,44,327.7,2021
1,2,Army West Point (FBS Independent),13,9-4,46,280.6,2021
2,3,Kent St. (MAC),14,7-7,36,248.6,2021
3,4,NIU (MAC),14,9-5,36,241.4,2021
4,5,North Texas (CUSA),13,6-7,32,233.5,2021
...,...,...,...,...,...,...,...
125,126,Stanford (Pac-12),12,3-9,13,86.9,2021
126,127,Purdue (Big Ten),13,9-4,7,84.2,2021
127,128,Arkansas St. (Sun Belt),12,2-10,4,82.4,2021
128,129,Nevada (Mountain West),13,8-5,13,77.2,2021


In [97]:
# Passing Offense for 2021
df_Passing_Offense21 = df_stats[32].drop(columns=['Metric', 'Pass Att', 'Pass Com', 'Pass Yds', 'Yds/Att', 'Yds/Comp'])
df_Passing_Offense21 = df_Passing_Offense21.rename(columns={'YPG': 'Pass YPG'})
df_Passing_Offense21 = df_Passing_Offense21.rename(columns={'Rank': 'Passing Rank'})
df_Passing_Offense21 = df_Passing_Offense21.rename(columns={'Pass TD': 'Passing TDs'})
df_Passing_Offense21['Season'] = 2021
df_Passing_Offense21

,Passing Rank,Team,G,W-L,Int,Passing TDs,Pass YPG,Season
0,1,Western Ky. (CUSA),14,9-5,12,63,433.7,2021
1,2,Virginia (ACC),12,6-6,12,31,392.6,2021
2,3,Ohio St. (Big Ten),13,11-2,8,46,380.9,2021
3,4,Mississippi St. (SEC),13,7-6,9,37,378.3,2021
4,5,Purdue (Big Ten),13,9-4,12,35,355.4,2021
...,...,...,...,...,...,...,...,...
125,126,Colorado (Pac-12),12,4-8,3,10,131.3,2021
126,127,New Mexico (Mountain West),12,3-9,8,10,114.1,2021
127,128,Air Force (Mountain West),13,10-3,4,8,95.5,2021
128,129,Army West Point (FBS Independent),13,9-4,3,11,94.8,2021


In [98]:
# Total Defense for 2021
df_Total_Defense21 = df_stats[33].drop(columns=['Metric', 'Plays', 'YDS', 'Yds/Play', 'Off TDs'])
df_Total_Defense21 = df_Total_Defense21.rename(columns={'YPG': 'Total Defense YPG'})
df_Total_Defense21 = df_Total_Defense21.rename(columns={'Rank': 'Defensive Rank'})
df_Total_Defense21['Season'] = 2021
df_Total_Defense21

,Defensive Rank,Team,G,W-L,Opp TDs,Total Defense YPG,Season
0,1,Wisconsin (Big Ten),13,9-4,25,239.1,2021
1,2,Georgia (SEC),15,14-1,16,268.9,2021
2,3,Minnesota (Big Ten),13,9-4,28,278.8,2021
3,4,Air Force (Mountain West),13,10-3,34,296.5,2021
4,5,Oklahoma St. (Big 12),14,12-2,30,297.9,2021
...,...,...,...,...,...,...,...
125,126,Kansas (Big 12),12,2-10,68,486.8,2021
126,127,New Mexico St. (FBS Independent),12,2-10,64,488.5,2021
127,128,FIU (CUSA),12,1-11,62,491.3,2021
128,129,Arkansas St. (Sun Belt),12,2-10,62,505.8,2021


In [99]:
# Rushing Defense for 2021
df_Rushing_Defense21 = df_stats[34].drop(columns=['Metric', 'Opp Rush', 'Opp Rush Yds', 'Yds/Rush'])
df_Rushing_Defense21 = df_Rushing_Defense21.rename(columns={'YPG': 'Opp Rush YPG'})
df_Rushing_Defense21 = df_Rushing_Defense21.rename(columns={'Rank': 'Rushing Defensive Rank'})
df_Rushing_Defense21['Season'] = 2021
df_Rushing_Defense21

,Rushing Defensive Rank,Team,G,W-L,Opp Rush TDs,Opp Rush YPG,Season
0,1,Wisconsin (Big Ten),13,9-4,10,64.8,2021
1,2,Georgia (SEC),15,14-1,3,78.9,2021
2,3,San Diego St. (Mountain West),14,12-2,7,80.3,2021
3,4,Alabama (SEC),15,13-2,9,86.0,2021
4,5,Oklahoma St. (Big 12),14,12-2,9,87.6,2021
...,...,...,...,...,...,...,...
125,126,Massachusetts (FBS Independent),12,1-11,43,235.4,2021
126,127,Stanford (Pac-12),12,3-9,33,235.7,2021
127,128,Akron (MAC),12,2-10,32,248.9,2021
128,129,Kansas (Big 12),12,2-10,40,249.9,2021


In [100]:
# Passing Defense for 2021
df_Passing_Defense21 = df_stats[35].drop(columns=['Metric', 'Opp Cpl', 'Opp Pass', 'Opp Pass <br/>Yds', 'Yds/Att', 'Yds/Comp'])
df_Passing_Defense21 = df_Passing_Defense21.rename(columns={'YPG': 'Opp Pass YPG'})
df_Passing_Defense21 = df_Passing_Defense21.rename(columns={'Rank': 'Passing Defensive Rank'})
df_Passing_Defense21 = df_Passing_Defense21.rename(columns={'Opp Pass <br/>TDs': 'Opp Pass TDs'})
df_Passing_Defense21['Season'] = 2021
df_Passing_Defense21

,Passing Defensive Rank,Team,G,W-L,Opp Pass TDs,Opp Pass YPG,Season
0,1,Washington (Pac-12),12,4-8,6,143.3,2021
1,2,Cincinnati (AAC),14,13-1,13,169.2,2021
2,3,Boston College (ACC),12,6-6,14,173.5,2021
3,4,Wisconsin (Big Ten),13,9-4,9,174.3,2021
4,5,Temple (AAC),12,3-9,25,177.9,2021
...,...,...,...,...,...,...,...
125,126,Ga. Southern (Sun Belt),12,3-9,28,289.5,2021
126,127,Hawaii (Mountain West),13,6-7,24,290.0,2021
127,128,New Mexico St. (FBS Independent),12,2-10,32,291.0,2021
128,129,Duke (ACC),12,3-9,32,311.8,2021


In [101]:
# Turnover Margin for 2021
df_Turnover_Margin21 = df_stats[36].drop(columns=['Metric', "Opp Int", "Turn Gain", "Int", "Turn Lost", "Avg"])
df_Turnover_Margin21 = df_Turnover_Margin21.rename(columns={'Fum Rec': 'Fumbles Recovered'})
df_Turnover_Margin21 = df_Turnover_Margin21.rename(columns={'Fum Lost': 'Fumbles Lost'})
df_Turnover_Margin21 = df_Turnover_Margin21.rename(columns={'Margin': 'Turnover Margin'})
df_Turnover_Margin21 = df_Turnover_Margin21.rename(columns={'Rank': 'TM Rank'})
df_Turnover_Margin21['Season'] = 2021
df_Turnover_Margin21

,TM Rank,Team,G,W-L,Fumbles Recovered,Fumbles Lost,Turnover Margin,Season
0,1,Nevada (Mountain West),13,8-5,13,1,17,2021
1,2,Louisiana (Sun Belt),14,13-1,13,4,15,2021
2,3,Baylor (Big 12),14,12-2,8,7,13,2021
3,3,Kent St. (MAC),14,7-7,9,6,13,2021
4,5,Cincinnati (AAC),14,13-1,11,9,12,2021
...,...,...,...,...,...,...,...,...
125,126,Kentucky (SEC),13,10-3,3,10,-11,2021
126,126,UTEP (CUSA),13,7-6,5,10,-11,2021
127,128,Indiana (Big Ten),12,2-10,4,7,-13,2021
128,129,FIU (CUSA),12,1-11,5,10,-14,2021


In [102]:
# Fewest Penalties Per Game for 2021
df_Penalties21 = df_stats[37].drop(columns=["Metric"])
df_Penalties21 = df_Penalties21.rename(columns={'PenPerGame': 'Penalties Per Game'})
df_Penalties21 = df_Penalties21.rename(columns={'Rank': 'PPG Rank'})
df_Penalties21['Season'] = 2021
df_Penalties21

,PPG Rank,Team,G,W-L,Penalties,Penalties Per Game,Season
0,1,ULM (Sun Belt),12,4-8,44,3.67,2021
1,2,Army West Point (FBS Independent),13,9-4,50,3.85,2021
2,2,Eastern Mich. (MAC),13,7-6,50,3.85,2021
3,4,Iowa (Big Ten),14,10-4,57,4.07,2021
4,5,Kansas (Big 12),12,2-10,49,4.08,2021
...,...,...,...,...,...,...,...
125,126,Arizona (Pac-12),12,1-11,105,8.75,2021
126,127,Arizona St. (Pac-12),13,8-5,114,8.77,2021
127,128,UAB (CUSA),13,9-4,115,8.85,2021
128,129,Ole Miss (SEC),13,10-3,118,9.08,2021


In [103]:
# Fewest Penalty Yards Per Game for 2021
df_Penalties_YPG21 = df_stats[38].drop(columns=["Metric", "Penalties", "PYards"])
df_Penalties_YPG21 = df_Penalties_YPG21.rename(columns={'YPG': 'Penalty YPG'})
df_Penalties_YPG21 = df_Penalties_YPG21.rename(columns={'Rank': 'Penalty YPG Rank'})
df_Penalties_YPG21['Season'] = 2021
df_Penalties_YPG21

,Penalty YPG Rank,Team,G,W-L,Penalty YPG,Season
0,1,ULM (Sun Belt),12,4-8,32.00,2021
1,2,Kansas (Big 12),12,2-10,33.67,2021
2,3,LSU (SEC),13,6-7,34.08,2021
3,4,Eastern Mich. (MAC),13,7-6,34.46,2021
4,5,Iowa (Big Ten),14,10-4,36.07,2021
...,...,...,...,...,...,...
125,126,Southern California (Pac-12),12,4-8,75.00,2021
126,127,Ole Miss (SEC),13,10-3,75.62,2021
127,128,Arizona St. (Pac-12),13,8-5,78.46,2021
128,129,UAB (CUSA),13,9-4,85.92,2021


In [104]:
# Time of Possession for 2021
df_TOP21 = df_stats[39].drop(columns=["Metric", "TOP"])
df_TOP21 = df_TOP21.rename(columns={'AvgTOP': 'Avg Time of Possession'})
df_TOP21 = df_TOP21.rename(columns={'Rank': 'ATP Rank'})

df_TOP21['Season'] = 2021
df_TOP21

,ATP Rank,Team,G,W-L,Avg Time of Possession,Season
0,1,Air Force (Mountain West),13,10-3,36:35,2021
1,2,Army West Point (FBS Independent),13,9-4,35:28,2021
2,3,Minnesota (Big Ten),13,9-4,35:25,2021
3,4,Western Mich. (MAC),13,8-5,35:16,2021
4,5,Navy (AAC),12,4-8,34:48,2021
...,...,...,...,...,...,...
125,126,Memphis (AAC),12,6-6,26:39,2021
126,127,FIU (CUSA),12,1-11,26:35,2021
127,128,San Jose St. (Mountain West),12,5-7,26:28,2021
128,129,UConn (FBS Independent),12,1-11,25:49,2021


In [105]:
# MERGE NCAA STATS 2024 DATA
ncaa_stats_24 = pd.merge(df_Total_Offense, df_Rushing_Offense, on=['Team',"G", "W-L", "Season"])
ncaa_stats_24 = pd.merge(ncaa_stats_24, df_Passing_Offense, on=['Team',"G", "W-L", "Season"])
ncaa_stats_24 = pd.merge(ncaa_stats_24, df_Total_Defense, on=['Team',"G", "W-L", "Season"])
ncaa_stats_24 = pd.merge(ncaa_stats_24, df_Rushing_Defense, on=['Team',"G", "W-L", "Season"])
ncaa_stats_24 = pd.merge(ncaa_stats_24, df_Passing_Defense, on=['Team',"G", "W-L", "Season"])
ncaa_stats_24 = pd.merge(ncaa_stats_24, df_Turnover_Margin, on=['Team',"G", "W-L", "Season"])
ncaa_stats_24 = pd.merge(ncaa_stats_24, df_Penalties, on=['Team',"G", "W-L", "Season"])
ncaa_stats_24 = pd.merge(ncaa_stats_24, df_Penalties_YPG, on=['Team',"G", "W-L", "Season"])
ncaa_stats_24 = pd.merge(ncaa_stats_24, df_TOP, on=['Team',"G", "W-L", "Season"])

ncaa_stats_24

,Offensive Rank,Team,G,W-L,Offense TDs,Total Offense YPG,Season,Rushing Rank,Rushing TDs,Rush YPG,...,Fumbles Recovered,Fumbles Lost,Turnover Margin,PPG Rank,Penalties,Penalties Per Game,Penalty YPG Rank,Penalty YPG,TOP Rank,Avg Time of Possession
0,1,Miami (FL) (ACC),13,10-3,71,537.2,2024,31,30,188.9,...,4,8,2,95,86,6.62,119,67.54,8,32:43
1,2,Ole Miss (SEC),13,10-3,60,526.5,2024,44,29,175.7,...,11,6,10,111,94,7.23,127,72.00,108,28:23
2,3,North Texas (The American),13,6-7,57,488.8,2024,69,24,160.6,...,5,8,-6,104,91,7.00,98,59.46,128,26:47
3,4,New Mexico (Mountain West),12,5-7,49,484.3,2024,2,37,253.6,...,7,7,-9,133,112,9.33,132,81.50,98,28:53
4,5,Texas St. (Sun Belt),13,8-5,58,476.9,2024,15,27,208.2,...,9,9,-5,100,88,6.77,114,64.15,34,31:12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,129,Michigan (Big Ten),13,8-5,30,286.2,2024,73,18,157.2,...,6,7,-2,5,52,4.00,14,39.54,20,31:54
129,130,Northwestern (Big Ten),12,4-8,19,284.4,2024,122,12,98.5,...,6,7,-2,16,57,4.75,11,38.25,81,29:33
130,131,Southern Miss. (Sun Belt),12,1-11,20,283.5,2024,118,13,108.6,...,4,12,-19,74,72,6.00,48,48.25,118,27:23
131,132,Florida St. (ACC),12,2-10,19,270.3,2024,128,8,89.9,...,2,9,-16,55,67,5.58,45,47.92,121,27:11


In [106]:
ncaa_stats_23 = pd.merge(df_Total_Offense23, df_Rushing_Offense23, on=['Team',"G", "W-L", "Season"])
ncaa_stats_23 = pd.merge(ncaa_stats_23, df_Passing_Offense23, on=['Team',"G", "W-L", "Season"])
ncaa_stats_23 = pd.merge(ncaa_stats_23, df_Total_Defense23, on=['Team',"G", "W-L", "Season"])
ncaa_stats_23 = pd.merge(ncaa_stats_23, df_Rushing_Defense23, on=['Team',"G", "W-L", "Season"])
ncaa_stats_23 = pd.merge(ncaa_stats_23, df_Passing_Defense23, on=['Team',"G", "W-L", "Season"])
ncaa_stats_23 = pd.merge(ncaa_stats_23, df_Turnover_Margin23, on=['Team',"G", "W-L", "Season"])
ncaa_stats_23 = pd.merge(ncaa_stats_23, df_Penalties23, on=['Team',"G", "W-L", "Season"])
ncaa_stats_23 = pd.merge(ncaa_stats_23, df_Penalties_YPG23, on=['Team',"G", "W-L", "Season"])
ncaa_stats_23 = pd.merge(ncaa_stats_23, df_TOP23, on=['Team',"G", "W-L", "Season"])
ncaa_stats_23

,Offensive Rank,Team,G,W-L,Offense TDs,Total Offense YPG,Season,Rushing Rank,Rushing TDs,Rush YPG,...,Fumbles Recovered,Fumbles Lost,Turnover Margin,PPG Rank,Penalties,Penalties Per Game,Penalty YPG Rank,Penalty YPG,TOP Rank,Avg Time of Possession
0,1,LSU (SEC),13,10-3,78,543.5,2023,10,34,204.5,...,4,3,6,71,79,6.08,67,51.69,94,28:51
1,2,Oregon (Pac-12),14,12-2,82,531.4,2023,29,33,184.5,...,6,2,11,98,93,6.64,106,60.57,41,31:03
2,3,Oklahoma (Big 12),13,10-3,66,507.0,2023,34,32,182.2,...,6,11,6,104,88,6.77,97,58.46,89,29:04
3,4,Liberty (CUSA),14,13-1,71,499.1,2023,1,39,293.3,...,4,12,7,84,88,6.29,117,62.50,8,33:04
4,5,Georgia (SEC),14,13-1,69,496.5,2023,20,40,191.2,...,3,7,3,12,62,4.43,29,43.29,2,33:28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,126,Akron (MAC),12,2-10,22,278.6,2023,128,12,81.0,...,6,6,-9,108,83,6.92,104,60.08,74,29:33
126,127,East Carolina (AAC),12,2-10,19,277.3,2023,110,12,113.3,...,10,7,-5,64,70,5.83,76,54.83,98,28:41
127,128,Kent St. (MAC),12,1-11,19,270.4,2023,115,8,105.9,...,3,6,-3,41,65,5.42,43,46.17,95,28:48
128,129,Eastern Mich. (MAC),13,6-7,28,262.8,2023,112,18,110.8,...,11,6,1,81,81,6.23,103,59.77,90,29:02


In [107]:
ncaa_stats_22 = pd.merge(df_Total_Offense22, df_Rushing_Offense22,  on=['Team',"G", "W-L", "Season"])
ncaa_stats_22 = pd.merge(ncaa_stats_22, df_Passing_Offense22, on=['Team',"G", "W-L", "Season"])
ncaa_stats_22 = pd.merge(ncaa_stats_22, df_Total_Defense22, on=['Team',"G", "W-L", "Season"])
ncaa_stats_22 = pd.merge(ncaa_stats_22, df_Rushing_Defense22, on=['Team',"G", "W-L", "Season"])
ncaa_stats_22 = pd.merge(ncaa_stats_22, df_Passing_Defense22, on=['Team',"G", "W-L", "Season"])
ncaa_stats_22 = pd.merge(ncaa_stats_22, df_Turnover_Margin22, on=['Team',"G", "W-L", "Season"])
ncaa_stats_22 = pd.merge(ncaa_stats_22, df_Penalties22, on=['Team',"G", "W-L", "Season"])
ncaa_stats_22 = pd.merge(ncaa_stats_22, df_Penalties_YPG22, on=['Team',"G", "W-L", "Season"])
ncaa_stats_22 = pd.merge(ncaa_stats_22, df_TOP22, on=['Team',"G", "W-L", "Season"])
ncaa_stats_22

,Offensive Rank,Team,G,W-L,Offense TDs,Total Offense YPG,Season,Rushing Rank,Rushing TDs,Rush YPG,...,Fumbles Recovered,Fumbles Lost,Turnover Margin,PPG Rank,Penalties,Penalties Per Game,Penalty YPG Rank,Penalty YPG,TOP Rank,Avg Time of Possession
0,1,Tennessee (SEC),13,11-2,78,525.5,2022,26,40,199.5,...,10,8,10,128,105,8.08,123,69.62,130,25:02
1,2,Washington (Pac-12),13,11-2,65,515.8,2022,68,33,146.0,...,5,2,1,98,89,6.85,100,60.46,30,31:21
2,3,Southern California (Pac-12),14,11-3,73,506.6,2022,52,30,171.1,...,10,2,22,76,88,6.29,88,58.00,23,31:57
3,4,UCLA (Pac-12),13,9-4,65,503.6,2022,6,36,237.2,...,7,9,-3,82,84,6.46,60,52.46,114,27:40
4,5,Georgia (SEC),15,15-0,75,501.2,2022,19,44,205.3,...,7,11,1,12,66,4.40,31,44.93,7,33:37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,126,Rutgers (Big Ten),12,4-8,22,282.1,2022,94,11,128.2,...,5,6,-6,123,94,7.83,115,64.17,78,29:11
126,127,Colorado (Pac-12),12,1-11,21,281.3,2022,114,11,108.3,...,5,12,-11,37,64,5.33,44,48.25,119,27:24
127,128,Massachusetts (FBS Independent),12,1-11,14,265.8,2022,67,10,148.8,...,8,6,-3,119,91,7.58,121,66.67,14,32:26
128,129,Iowa (Big Ten),13,8-5,19,251.5,2022,123,12,94.8,...,8,10,6,6,53,4.08,5,32.69,104,28:20


In [108]:
ncaa_stats_21 = pd.merge(df_Total_Offense21, df_Rushing_Offense21, on=['Team',"G", "W-L", "Season"])
ncaa_stats_21 = pd.merge(ncaa_stats_21, df_Passing_Offense21, on=['Team',"G", "W-L", "Season"])
ncaa_stats_21 = pd.merge(ncaa_stats_21, df_Total_Defense21, on=['Team',"G", "W-L", "Season"])
ncaa_stats_21 = pd.merge(ncaa_stats_21, df_Rushing_Defense21, on=['Team',"G", "W-L", "Season"])
ncaa_stats_21 = pd.merge(ncaa_stats_21, df_Passing_Defense21, on=['Team',"G", "W-L", "Season"])
ncaa_stats_21 = pd.merge(ncaa_stats_21, df_Turnover_Margin21, on=['Team',"G", "W-L", "Season"])
ncaa_stats_21 = pd.merge(ncaa_stats_21, df_Penalties21, on=['Team',"G", "W-L", "Season"])
ncaa_stats_21 = pd.merge(ncaa_stats_21, df_Penalties_YPG21, on=['Team',"G", "W-L", "Season"])
ncaa_stats_21 = pd.merge(ncaa_stats_21, df_TOP21, on=['Team',"G", "W-L", "Season"])
ncaa_stats_21

,Offensive Rank,Team,G,W-L,Offense TDs,Total Offense YPG,Season,Rushing Rank,Rushing TDs,Rush YPG,...,Fumbles Recovered,Fumbles Lost,Turnover Margin,PPG Rank,Penalties,Penalties Per Game,Penalty YPG Rank,Penalty YPG,ATP Rank,Avg Time of Possession
0,1,Ohio St. (Big Ten),13,11-2,70,561.5,2021,47,24,180.6,...,8,3,9,81,85,6.54,91,59.23,81,29:23
1,2,Western Ky. (CUSA),14,9-5,76,535.3,2021,121,13,101.6,...,10,8,11,92,95,6.79,69,54.14,113,27:49
2,3,Virginia (ACC),12,6-6,52,515.8,2021,105,21,123.2,...,7,6,-3,75,77,6.42,79,56.75,40,30:38
3,4,Kent St. (MAC),14,7-7,57,495.2,2021,3,36,248.6,...,9,6,13,119,110,7.86,122,71.29,125,26:41
4,5,Coastal Carolina (Sun Belt),13,11-2,69,494.7,2021,6,36,228.8,...,6,6,2,40,70,5.38,59,51.23,24,31:18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,126,Navy (AAC),12,4-8,25,282.3,2021,9,19,225.5,...,6,5,5,9,53,4.42,6,36.17,5,34:48
126,127,Southern Miss. (CUSA),12,3-9,22,266.3,2021,109,8,120.8,...,13,14,-7,124,98,8.17,101,62.50,46,30:29
127,128,UConn (FBS Independent),12,1-11,19,265.0,2021,122,8,100.8,...,4,5,-8,88,81,6.75,49,49.17,129,25:49
128,129,Colorado (Pac-12),12,4-8,22,257.4,2021,98,12,126.2,...,4,4,6,73,76,6.33,78,56.42,122,27:12


In [117]:
ncaa_stats_final = pd.concat([ncaa_stats_24, ncaa_stats_23, ncaa_stats_22, ncaa_stats_21], ignore_index=True)
ncaa_stats_final

,Offensive Rank,Team,G,W-L,Offense TDs,Total Offense YPG,Season,Rushing Rank,Rushing TDs,Rush YPG,...,Fumbles Lost,Turnover Margin,PPG Rank,Penalties,Penalties Per Game,Penalty YPG Rank,Penalty YPG,TOP Rank,Avg Time of Possession,ATP Rank
0,1,Miami (FL) (ACC),13,10-3,71,537.2,2024,31,30,188.9,...,8,2,95,86,6.62,119,67.54,8,32:43,NaN
1,2,Ole Miss (SEC),13,10-3,60,526.5,2024,44,29,175.7,...,6,10,111,94,7.23,127,72.00,108,28:23,NaN
2,3,North Texas (The American),13,6-7,57,488.8,2024,69,24,160.6,...,8,-6,104,91,7.00,98,59.46,128,26:47,NaN
3,4,New Mexico (Mountain West),12,5-7,49,484.3,2024,2,37,253.6,...,7,-9,133,112,9.33,132,81.50,98,28:53,NaN
4,5,Texas St. (Sun Belt),13,8-5,58,476.9,2024,15,27,208.2,...,9,-5,100,88,6.77,114,64.15,34,31:12,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518,126,Navy (AAC),12,4-8,25,282.3,2021,9,19,225.5,...,5,5,9,53,4.42,6,36.17,NaN,34:48,5.0
519,127,Southern Miss. (CUSA),12,3-9,22,266.3,2021,109,8,120.8,...,14,-7,124,98,8.17,101,62.5,NaN,30:29,46.0
520,128,UConn (FBS Independent),12,1-11,19,265.0,2021,122,8,100.8,...,5,-8,88,81,6.75,49,49.17,NaN,25:49,129.0
521,129,Colorado (Pac-12),12,4-8,22,257.4,2021,98,12,126.2,...,4,6,73,76,6.33,78,56.42,NaN,27:12,122.0


In [118]:
ncaa_stats_final[['Team','Conference']] = ncaa_stats_final['Team'].str.rsplit('(', n = 1, expand = True)
ncaa_stats_final['Conference'] = ncaa_stats_final['Conference'].str.replace(')', '')
ncaa_stats_final.head(3)

,Offensive Rank,Team,G,W-L,Offense TDs,Total Offense YPG,Season,Rushing Rank,Rushing TDs,Rush YPG,...,Turnover Margin,PPG Rank,Penalties,Penalties Per Game,Penalty YPG Rank,Penalty YPG,TOP Rank,Avg Time of Possession,ATP Rank,Conference
0,1,Miami (FL),13,10-3,71,537.2,2024,31,30,188.9,...,2,95,86,6.62,119,67.54,8,32:43,NaN,ACC
1,2,Ole Miss,13,10-3,60,526.5,2024,44,29,175.7,...,10,111,94,7.23,127,72.00,108,28:23,NaN,SEC
2,3,North Texas,13,6-7,57,488.8,2024,69,24,160.6,...,-6,104,91,7.00,98,59.46,128,26:47,NaN,The American


In [119]:
ncaa_stats_final[['Wins', 'Losses']] = ncaa_stats_final['W-L'].str.split('-', expand=True)
ncaa_stats_final['Wins'] = ncaa_stats_final['Wins'].astype(int)
ncaa_stats_final['Losses'] = ncaa_stats_final['Losses'].astype(int)
ncaa_stats_final = ncaa_stats_final.drop(columns='W-L')
ncaa_stats_final.head(3)

,Offensive Rank,Team,G,Offense TDs,Total Offense YPG,Season,Rushing Rank,Rushing TDs,Rush YPG,Passing Rank,...,Penalties,Penalties Per Game,Penalty YPG Rank,Penalty YPG,TOP Rank,Avg Time of Possession,ATP Rank,Conference,Wins,Losses
0,1,Miami (FL),13,71,537.2,2024,31,30,188.9,3,...,86,6.62,119,67.54,8,32:43,NaN,ACC,10,3
1,2,Ole Miss,13,60,526.5,2024,44,29,175.7,2,...,94,7.23,127,72.00,108,28:23,NaN,SEC,10,3
2,3,North Texas,13,57,488.8,2024,69,24,160.6,4,...,91,7.00,98,59.46,128,26:47,NaN,The American,6,7


In [120]:
ncaa_stats_final.to_csv("NCAA_stats.csv", index=False)